In [ ]:
import pandas as pd
import dash
from dash import dcc, html
import plotly.express as px
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import pearsonr
import numpy as np
import random
from deap import base, creator, tools, algorithms
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Carrega os dados
dados = pd.read_csv('Dados Brutos-PA-CFCT-comCNR.csv', parse_dates=[0])
dados = dados.drop(columns=['DATA'])
X = dados.drop(columns=['CFCT'])
y = dados['CFCT']

In [ ]:
# Treina um modelo de Regressão de Floresta Aleatória
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X, y)

In [ ]:
# Calcula a importância das características
importancias = rf.feature_importances_
importancias_caracteristicas = pd.DataFrame({'Atributo': X.columns, 'Importância': importancias})
importancias_caracteristicas = importancias_caracteristicas.sort_values(by='Importância', ascending=False)
atributos_mais_importantes_rf = importancias_caracteristicas['Atributo'].to_list()


In [ ]:
# Gera o gráfico da importância das características
plt.figure(figsize=(10, 6))
plt.bar(range(len(atributos_mais_importantes_rf)), importancias_caracteristicas['Importância'], tick_label=atributos_mais_importantes_rf)
plt.xlabel('Atributos')
plt.ylabel('Importância')
plt.title('Importância dos Atributos (Random Forest)')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

In [ ]:
# Calcula as correlações de Pearson
correlacoes = [pearsonr(dados[attr], dados['CFCT'])[0] for attr in X.columns]
correlacoes_df = pd.DataFrame({'Atributo': X.columns, 'Correlação de Pearson': correlacoes})
correlacoes_df = correlacoes_df.sort_values(by='Correlação de Pearson', ascending=False)
atributos_mais_importantes_correlacao = correlacoes_df['Atributo'].to_list()

In [ ]:
# Gera o gráfico da correlação de Pearson
plt.figure(figsize=(10, 6))
plt.bar(range(len(atributos_mais_importantes_correlacao)), correlacoes_df['Correlação de Pearson'], tick_label=atributos_mais_importantes_correlacao)
plt.xlabel('Atributos')
plt.ylabel('Correlação de Pearson')
plt.title('Correlação de Pearson entre Atributos e CFCT')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

In [ ]:
# Configuração do Algoritmo Genético
todos_atributos = X.columns.tolist()

In [ ]:
def avaliacao(individual):
    atributos_selecionados = [attr for attr, selecionado in zip(todos_atributos, individual) if selecionado]
    valor_avaliacao = np.sum(np.random.rand(len(atributos_selecionados)))  # Exemplo simples de função de avaliação
    return (valor_avaliacao,)

In [ ]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)
toolbox = base.Toolbox()
toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=len(todos_atributos))
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", avaliacao)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)

In [ ]:
NGEN = 100  # Número máximo de gerações
CXPB = 0.9   # Probabilidade de crossover
MUTPB = 0.2  # Probabilidade de mutação
populacao = toolbox.population(n=100)  # Tamanho da população


In [ ]:
melhores_valores_fitness = []
melhores_individuos = []
for geracao in range(NGEN):
    fitnesses = list(map(toolbox.evaluate, populacao))
    for ind, fit in zip(populacao, fitnesses):
        ind.fitness.values = fit

    melhor_individuo = tools.selBest(populacao, k=1)[0]
    melhores_valores_fitness.append(melhor_individuo.fitness.values)
    melhores_individuos.append(melhor_individuo)

    descendentes = algorithms.varAnd(populacao, toolbox, cxpb=CXPB, mutpb=MUTPB)
    fitnesses = list(map(toolbox.evaluate, descendentes))
    for ind, fit in zip(descendentes, fitnesses):
        ind.fitness.values = fit
    populacao = toolbox.select(descendentes, k=len(populacao))

In [ ]:
melhores_valores_fitness = np.array(melhores_valores_fitness)
geracoes = np.arange(1, NGEN + 1)

In [ ]:
# Gera o gráfico da convergência do Algoritmo Genético
plt.figure(figsize=(10, 6))
plt.plot(geracoes, melhores_valores_fitness, marker='o')
plt.xlabel('Geração')
plt.ylabel('Melhor Valor de Fitness')
plt.title('Convergência do Algoritmo Genético')
plt.grid(True)
plt.show()

In [ ]:
melhor_individuo = tools.selBest(populacao, k=1)[0]
atributos_selecionados = [attr for attr, selecionado in zip(todos_atributos, melhor_individuo) if selecionado]
print("Melhor conjunto de atributos:", atributos_selecionados)

In [ ]:
# Gera o heatmap da matriz de correlação
matriz_correlacao = dados.corr()
plt.figure(figsize=(15, 9))
sns.heatmap(matriz_correlacao, annot=True, cmap="coolwarm", fmt=".2f", linewidths=.5)
plt.title('Matriz de Correlação com Heatmap')
plt.show()

In [ ]:
# Criar um aplicativo Dash
# app = dash.Dash(__name__)

# # Layout do aplicativo
# app.layout = html.Div([
#     # Gráfico de barras para a importância das características no RandomForest
#     dcc.Graph(
#         id='importancia-atributos-rf',
#         figure=px.bar(
#             # feature_importances,
#             x='Atributo',
#             y='Importância',
#             title='Importância dos Atributos (Random Forest)',
#             labels={'Importância': 'Importância', 'Atributo': 'Atributos'},
#             height=600
#         )
#     ),

#     # Gráfico de barras para a correlação de Pearson entre atributos e CFCT
#     dcc.Graph(
#         id='correlacao-pearson',
#         figure=px.bar(
#             correlacoes_df,
#             x='Atributo',
#             y='Correlação de Pearson',
#             title='Correlação de Pearson entre Atributos e CFCT',
#             labels={'Correlação de Pearson': 'Correlação de Pearson', 'Atributo': 'Atributos'},
#             height=600
#         )
#     ),

#     # Gráfico de heatmap para a matriz de correlação
#     dcc.Graph(
#         id='heatmap-correlacao',
#         figure=px.imshow(
#             data.corr(),
#             color_continuous_scale='coolwarm',
#             labels={'index': 'Atributos', 'columns': 'Atributos'},
#             title='Matriz de Correlação com Heatmap'
#         )
#     )
# ])

# # Executar o aplicativo Dash
# if __name__ == '__main__':
#     app.run_server(debug=True)